In [ ]:
from analysis.paper import *

In [ ]:
!pip install sqlalchemy

In [ ]:
ags = data.load()

In [ ]:
from io import BytesIO
import matplotlib.pyplot as plt

ax = plot_frontiers(ags)
fig = ax.draw()

bs = BytesIO()
fig.savefig(bs, bbox_inches='tight', format='png')

plt.close(fig)

In [ ]:
import dropbox

token = 'ngDMVVaqBG4AAAAAAAAAAdx4TlZOCYFJQhQnXCi85cf_gVtkAij8M3Nn_-BgIwbF'

path = 'frontier2.png'

dbx = dropbox.Dropbox(token)

r = dbx.files_upload(
    f=bs.getvalue(), 
    path=f'/Apps/Overleaf/boardlaw/images/{path}',
    mode=dropbox.files.WriteMode.overwrite)

In [ ]:
plot.overleaf(ax, 'frontier3.png')

In [ ]:
list(Path(path).parent.glob('*'))

In [ ]:
import requests

In [ ]:
type(ax)